<a href="https://colab.research.google.com/github/EugeneFrancisco/Number-Theory-Playground/blob/main/Finding_Generators.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Finding generators in $\mathbb{F}_{p^2}$, where $p$ is a prime.

Finding generators in $\mathbb{F}_{p^2}$ can be challenging; in general finding the order of specific elements in quotient rings is a hard task which, by hand, requires guessing and checking. Here, I've written some functions to check the order of elements in finite fields of size $p^2$.


I choose to abstract polynomials as vectors, where a polynomial of degree $d$ is represented by a vector $f$ of size $d+1$ and the $f[i]$ is the coefficient of the term with degree $d-i$. I.e., coefficients go from higher degree to smaller degree.

## Some basic utility for multiplying and adding polynomials in $\mathbb{F}_{p}[x]$.

In [1]:
'''
Given product1 and product2 in Fp[x] (as vectors), where p is the base, returns a vector which represents the
polynomial product1*product2.

product1 & product2 are vectors. For example, the polynomial x^3+x would be represented as [1,0,1,0]

The return value is a vector which corresponds to the fully expanded multiplied form of
product1 * product2.

base is the base of the ring used to form the polynomial.
'''
def mult(product1, product2, prime):
    result = [0]*(len(product1)+len(product2)-1)
    for i, term1 in enumerate(product1):
        deg1 = len(product1)-1-i
        for j, term2 in enumerate(product2):
            deg2 = len(product2)-1-j
            resultIndex = len(result) - 1 - (deg1+deg2)  #index in the result vector this shows up at
            result[resultIndex] += (term1 * term2)
            if prime != 0:
                result[resultIndex] %= prime

    if prime != 0:
        for i, term in enumerate(result):
            result[i] = term%prime

    return result

def findInverses(prime):
    inverseDictionary = {1:1, prime-1: prime-1}
    for i in range(2, prime-1):
        # use Fermat's little Theorem
        inverseDictionary[i] = (i**(prime-2))%prime
    return inverseDictionary

'''
Returns a trimmed polynomial without the leading zeroes.
'''
def trimPolynomial(polynomial):
    result = []
    j = 0
    while (j != len(polynomial) and polynomial[j] == 0):
        j +=1
    return polynomial[j:]

'''
Given summand1, summand2, and a prime, adds the polynomials (represented as vectors) of
summand1 and summand2 together.
'''
def addPolynomials(summand1, summand2, prime):
    if (len(summand1) > len(summand2)):
        result = summand1
        other = summand2
    else:
        result = summand2
        other = summand1
    i = len(result)-1
    for term in reversed(other):
        # traverse term by term from lowest to highest degree of other
        result[i] += term
        i-=1

    if (prime != 0):
        for i in range(len(result)):
            result[i] %= prime

    result = trimPolynomial(result)
    return result

## Utility for dividing polynomials in $\mathbb{F}_p[x]$ and finding residue classes in $\mathbb{F}_p[x]/(f)$, where $f$ is some irreducible over the polynomial ring.

In [10]:
'''
Given a tuple of the form (coeff, degree) representing a monomial, converts the monomial into a polynomial
vector as used elsewhere
'''
def tupleToPoly(tuple):
    result = [0]*(tuple[1]+1)
    result[0] = tuple[0]
    return result

def deg(polynomial):
    return len(polynomial)-1

'''
Given a dividend and a divisor (both tuples of the form (coeff, degree)) returns the
quotient (same form). prime is a prime prime of the ring R.

inverses is a list of key value pairs of elements in the ring and their inverses
'''
def singleTermDivide(dividend, divisor, inverses, prime):
    if (divisor[1] > dividend[1]):
        raise ValueError("The degree of the divisor cannot be greater than the degree of the dividend")
    resultDeg = dividend[1]-divisor[1]
    resultCoeff = (dividend[0]*inverses[divisor[0]])%prime
    return (resultCoeff, resultDeg)

'''
Given a dividend and a divisor working in the prime, returns the
remainder, quotient where remainder and quotient are polynomials.

Recall the "norm" of polynomials is their degree. If the dividend has degree n and the
divisor has degree d, the quotient has degree n - d and the remainder has degree at most n.
'''

def findResidue(dividend, divisor, prime):
    quotient = [0]*(len(dividend)-len(divisor)+1)
    remainder = dividend
    leadingDivisorTuple = (divisor[0], deg(divisor))
    inverses = findInverses(prime)
    while (deg(divisor) <= deg(remainder)):
        # 1. Divide first term of remainder by first term of divisor (call this q_i)
        leadingRemainderTuple = (remainder[0], deg(remainder))
        firstQuotient = singleTermDivide(leadingRemainderTuple, leadingDivisorTuple, inverses, prime)
        # 2. Add this term to the quotient
        quotient[len(quotient)-firstQuotient[1]-1] = firstQuotient[0]
        # 3. Multiply q_i by the divisor and then subtract that from the remainder.
        subtractPoly = mult(divisor, tupleToPoly((firstQuotient[0]*-1, firstQuotient[1])), prime)
        # 4. The result is now the new remainder and repeat.
        remainder = addPolynomials(subtractPoly, remainder, prime)

    return remainder, quotient



## Putting it together.

Implementing the above utility to actually create some finite fields and find the order of certain elements.

In [2]:
'''
Given a prime p and an irreducible f over Fp, populates a dictionary with the list of
elements generated by each polynomial in the form

element: poly1, poly2, ... 1

The order of an element is then just the number of elements in that element's list (the element's coset)

*Note, the key's of the dictionary are tuples, not lists*
'''

def findOrders(prime, irreducible):

    cosetDictionary = {}

    # The number of coefficients of elements in the quotient ring is just the degree of the quotient ring
    numCoefficients = deg(irreducible)
    findOrdersRec([], numCoefficients, prime, cosetDictionary, irreducible)
    return cosetDictionary

'''
Recursive helper to create all possible combinations of polynomials
'''
def findOrdersRec(coeffList, howManyMore, prime, cosetDictionary, irreducible):
    if (howManyMore == 0):
        coeffListCopy = trimPolynomial(coeffList)
        if (len(coeffListCopy) == 0):
            return

        # Need to use hashable type for the dictionary.
        coeffListTuple = tuple(coeffListCopy)
        cosetDictionary[coeffListTuple] = findCoset(prime, irreducible, coeffListCopy)
        return

    for coeff in range(prime):
        coeffList.append(coeff)
        findOrdersRec(coeffList, howManyMore-1, prime, cosetDictionary, irreducible)
        coeffList.pop()

def findCoset(prime, irreducible, element):
    cosetList = [1]
    result = trimPolynomial(element)
    while (result != [1]):
        cosetList.append(result)
        result = mult(result, element, prime)
        result = findResidue(result, irreducible, prime)[0]

    return cosetList


# Try it!

As an example, in $\mathbb{F}_5[x]$ an irreducible polynomial is $x^2+2$ and a generator of $\mathbb{F}_5[x]/(x^2+2)$ is $x+1$.

In [ ]:
prime = input("Choose a prime p: ")
irreducibleTxt = input("Choose an irreducible in Fp[x]. List the coefficients one by one, seperated by commas: ")
irreducible = irreducibleTxt.split(",")
irreducible = [int(x) for x in irreducible]

print(irreducible)
cosetDictionary = findOrders(5, irreducible)

for poly in cosetDictionary:
    print("Polynomial: ", poly, "\nOrder: ", len(cosetDictionary[poly]), "\n")